In [5]:
from itertools import repeat

def sheet4():   # adding a column in sheet4 for main category and sub category
     colSheet4 = ["Main-Hospital","Sub-Hospital","Sub-Hospital","Sub-Hospital","Sub-Hospital","Sub-Hospital","Main-Residential-Care","Sub-Residential-Care","Sub-Residential-Care",
             "Main-Home","Sub-Home","Sub-Home","Main-Other",""]
     return colSheet4

def sheet5():  # adding a column in sheet5 for main category and sub category
    colSheet5 =["","Main-Hospital"]
    colSheet5.extend(repeat("Sub-Hospital",5))
    colSheet5.extend(repeat("Main-Residential-Care",1))
    colSheet5.extend(repeat("Sub-Residential-Care",2))
    colSheet5.extend(repeat("Main-Home",1))
    colSheet5.extend(repeat("Sub-Home",2))
    colSheet5.extend(repeat("Main-Other",1))
    colSheet5.extend(repeat("",1))
    return colSheet5

def sheet6():   # adding a column in sheet6 for main category and sub category
    colSheet6 =["Main-Endocrine/metabolic/nutritional diseases"]
    colSheet6.extend(repeat("Sub-Endocrine/metabolic/nutritional diseases",3))
    colSheet6.extend(repeat("Main-Heart/circulation diseases",1))
    colSheet6.extend(repeat("Sub-Heart/circulation diseases",8))
    colSheet6.extend(repeat("Main-Musculoskeletal diseases",1))
    colSheet6.extend(repeat("Sub-Musculoskeletal diseases",5))
    colSheet6.extend(repeat("Main-Neurological diseases",1))
    colSheet6.extend(repeat("Sub-Neurological diseases",16))
    colSheet6.extend(repeat("Main-Psychiatric/mood diseases",1))
    colSheet6.extend(repeat("Sub-Psychiatric/mood diseases",4))
    colSheet6.extend(repeat("Main-Pulmonary diseases",1))
    colSheet6.extend(repeat("Sub-Pulmonary diseases",2))
    colSheet6.extend(repeat("Main-Sensory diseases",1))
    colSheet6.extend(repeat("Sub-Sensory diseases",4))
    colSheet6.extend(repeat("Main-Other diseases",1))
    colSheet6.extend(repeat("Sub-Other diseases",6))
    colSheet6.extend(repeat("None of the above",1))
    colSheet6.extend(repeat("",1))
    return colSheet6
    
def sheet8():   # adding a column in sheet8 for main category and sub category
    colSheet8 =["Main-Relatively-intact"]
    colSheet8.extend(repeat("Sub-Relatively-intact",2))
    colSheet8.extend(repeat("Main-Mild/moderate",1))
    colSheet8.extend(repeat("Sub-Mild/moderate",2))
    colSheet8.extend(repeat("Main-Severe",1))
    colSheet8.extend(repeat("Sub-Severe",3))
    colSheet8.extend(repeat("",1))
    return colSheet8 

def sheet16():     # adding a column in sheet16 for main category and sub category
    colSheet16 =["Main-Therapies"]
    colSheet16.extend(repeat("Sub-Therapies",6))
    colSheet16.extend(repeat("Main-Special treatments",1))
    colSheet16.extend(repeat("Sub-Special treatments",12))
    colSheet16.extend(repeat("Main-Programs",1))
    colSheet16.extend(repeat("Sub-Programs",6))
    colSheet16.extend(repeat("",1))
    return colSheet16

In [7]:
import pandas as pd
import xlrd
writer = pd.ExcelWriter('resultFile.xlsx', engine='xlsxwriter')
PATH1 = 'ccrs-quick-stats-2016-2017-en.xlsx'
PATH2 = 'ccrs-quick-stats-2017-2018-en.xlsx'
PATH3 = 'ccrs-quick-stats-2018-2019-en.xlsx'
PATH4 = 'ccrs-quick-stats-2019-2020-en.xlsx'
PATH5 = 'ccrs-quick-stats-2020-2021-en.xlsx'
ALLPATH = [PATH1,PATH2,PATH3,PATH4,PATH5]  #**** all files' path , in case of a new dataset we need to add it's path here
for i in range(3, 23): 
    
    data=[]
    alldf = pd.DataFrame(data)
    for path in ALLPATH:
        filemain = pd.ExcelFile(path)  #when we have multiple sheet
        allsheets= filemain.sheet_names   
        df1 = pd.read_excel(filemain, sheet_name=i)  # reading sheet1
    
        if i<5:  #some sheets style are different so I seperated the first 2 sheets formula
           
                df1.columns = df1.iloc[3]   # set first row as a header
                df1.drop([0,1,2,3], inplace=True) # removing 3 first rows
           
        else:
         
            df1.columns = df1.iloc[4]   # set first row as a header
            df1.drop([0,1,2,3,4], inplace=True) # removing 4 first rows
    
        df1.dropna(how='all', axis=1 ,inplace=True)
        df1.dropna(how='any',thresh=2, inplace=True)  # drop all NAN after "Note"
        cols=[]
        Ontario = "ontario"
        Residential = "residential"
        Hospital = "hospital"
        
        for c in df1.columns:
            if Hospital not in c.lower() :  # get all columns which is not started with Hospital
                if  Residential not in c.lower():  # all first columns with no Hospital or Residential
                            cols.append(c) 
                else:
                    if Ontario in c.lower() :  # column with Residential and ontario
                        cols.append(c)# get the column with Residential Ontario
     
        df= df1[cols] # showing this column
        if path == PATH1:
            df = df.assign(Year='2016-2017')  #**** haveing year column, if we have a new data 2021-2022, we need to add one more for that
        elif path == PATH2:
            df = df.assign(Year='2017-2018')
        elif path == PATH3:
            df = df.assign(Year='2018-2019')
        elif path == PATH4:
            df = df.assign(Year='2019-2020')
        elif path == PATH5:
            df = df.assign(Year='2020-2021')
            
            
        if (i==6):  #adding category column to show main and sub category for some sheets
             df["Category"]= sheet4()
        elif i==7:
             df["Category"]= sheet5()
        elif i==8:
            df["Category"]= sheet6()
        elif i==10:
            df["Category"]= sheet8()
        elif i==18:
            df["Category"]= sheet16() 
            
        if path == PATH1:
            allColName= df.columns    # columns name may be different in other files, so we get columns name from the first file
        df.columns =  allColName       # and change all other columns from other files to that one
        alldf = pd.concat([alldf, df], axis=0)  # concatenate all years from sheet i 
        
    alldf.reset_index(drop=True, inplace=True) # set index from 0 , 1, 2,....
    alldf.columns.name=""  # removing number 3 at the top row

    alldf.to_excel(writer, sheet_name = allsheets[i], index = True)
    i = i+1
        
writer.save()